# Extracting `PoolCreated` events from Uniswap

This example shows how `evm_decode` and `evm_topic` can be used to find events and decode them. 

In [ ]:
from nozzle.client import Client
from nozzle.util import to_hex
import os.path

client = Client("grpc://127.0.0.1:1602")

# The address of the Uniswap factory contract
uniswap_factory = "1F98431c8aD98523631AE4a59f267346ea31F984"

# Adjust to a directory with the Uniswap v3 ABIs
abi_root = "/var/tmp/abis"
factory_path = os.path.join(abi_root, "factory.json")
pool_path = os.path.join(abi_root, "pool.json")

In [ ]:
client.get_sql("select count(*) logs_count from logs", read_all=True)

In [ ]:
client.get_sql("select count(*) blocks_count, min(block_num) first_block, max(block_num) latest_block from blocks", read_all=True)

In [ ]:
from nozzle.util import Abi
import time

# Load a JSON ABI and get the 'PoolCreated' event
factory = Abi(factory_path)

pool_created = factory.events["PoolCreated"]

query = """
select pc.block_num,
       pc.dec['pool'] as pool,
       pc.dec['token0'] as token0,
       pc.dec['token1'] as token1,
       pc.dec['fee'] as fee,
       pc.dec['tickSpacing'] as tick_spacing
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{1}') as dec
              from logs l
             where l.address = arrow_cast(x'{0}', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('{1}')) pc
""".format(uniswap_factory, pool_created.signature())

print(query)

start = time.time()
df = client.get_sql(query).map(to_hex)
elapsed = time.time() - start
print('time: ', elapsed, 's')
df.count()

In [ ]:
df.head(5)

In [ ]:
pool_abi = Abi(pool_path)
swap_ev = pool_abi.events["Swap"]

query = """
with pc as (
  select pc.block_num,
         pc.dec['pool'] as pool,
         pc.dec['token0'] as token0,
         pc.dec['token1'] as token1,
         pc.dec['fee'] as fee,
         pc.dec['tickSpacing'] as tick_spacing
    from (select l.block_num,
                 evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{1}') as dec
            from logs l
           where l.address = arrow_cast(x'{0}', 'FixedSizeBinary(20)')
             and l.topic0 = evm_topic('{1}')) as pc)
select sw.block_num, sw.tx_hash,
       sw.dec['sender'] as sender,
       sw.dec['recipient'] as recipient,
       sw.dec['amount0'] as amount0,
       sw.dec['amount1'] as amount1,
       sw.dec['sqrtPriceX96'] as sqrt_price_x96,
       sw.dec['liquidity'] as liquidity,
       sw.dec['tick'] as tick
  from (select sl.block_num, sl.tx_hash,
               evm_decode(sl.topic1, sl.topic2, sl.topic3, sl.data, '{2}') as dec
          from logs sl, pc
         where sl.address = pc.pool
           and sl.block_num >= pc.block_num
           and sl.topic0 = evm_topic('{2}')) as sw
""".format(uniswap_factory, pool_created.signature(), swap_ev.signature())

print(query)

start = time.time()
df = client.get_sql(query).map(to_hex)
elapsed = time.time() - start
print('time: ', elapsed, 's')
df.count()

In [ ]:
df.head(5)

In [ ]:
df.head(5).to_csv("/tmp/swaps.csv")